In [16]:
import vk_api
import requests
import pandas as pd
import time
import re
import json
from io import StringIO

In [17]:
CLIENT_ID = '7481982' #ID приложения
link = 'https://oauth.vk.com/authorize?client_id=' + CLIENT_ID + '&scope=friends&response_type=token&v=5.103'
link

'https://oauth.vk.com/authorize?client_id=7481982&scope=friends&response_type=token&v=5.103'

In [18]:
base_url = "https://api.vk.com/method/%s"
access_token = '0952ab82b3c453e22025005637fed4a7777d3f4d1b5efd6f8e6136a568fc96af510c014d24961b9ca3cb4' 

def call(url, params):
    return requests.get(url, params=params)

In [19]:
def users_search(school, birth_year, sex):
        params = {
            'v': '5.103',
            'school': school,
            'sex': sex,
            'birth_year': birth_year,
            'count': 1000
        }
    
    
        params = dict(params, **{
            "access_token": access_token
        })
        #print(params)
        #print(base_url % "users.search", params)
        return call(base_url % "users.search", params)

In [20]:
with open('schools.json') as f:
    text = f.read()
    schools_dict = json.loads(text)

In [30]:
top10_schools = list(schools_dict.values())[49:59]
for school in top10_schools:
    print(school)

1712330
14172
3707
7326
277
835
7030
7416
43831
14072


In [31]:
birth_year = [i for i in range(1996, 2007)]
sexes = [1, 2]
search_params = []
for year in birth_year:
    for sex in sexes:
        search_params.append((year, sex))
search_params[0]

(1996, 1)

In [32]:
def search_for_schoolers(school, search_params, param_ind=0):
    all_schoolers = {}
    for params in search_params[param_ind:]:
        q = 0
        year, sex = params
        if year not in all_schoolers:
            all_schoolers[year] = {}
        error_counter = 0
        all_schoolers[year][sex] = []
        time.sleep(3)
        search_results = users_search(school, year, sex).json()
        while 'error' in search_results:
            if error_counter < 3:
                time.sleep(3) 
                error_counter +=1 
                search_results = users_search(school, year, sex).json()
            if 3 <= error_counter < 10:
                time.sleep(60) 
                error_counter +=1 
                search_results = users_search(school, year, sex).json()
            if error_counter == 10:
                print('Error occure more than 10 times in a row, will be sleeping for 40 minutes')
                print(search_results)
                time.sleep(2400)
                error_counter +=1
                search_results = users_search(school, year, sex).json()
            if error_counter > 10:
                safe_info = {'school': school, 'year': year, 'sex': sex}
                file_name = "schoolers_safe_point" + str(school) + ".json" 
                with open(file_name, "w") as write_file:
                    json.dump(all_schoolers, write_file)
                with open('start_file', "w") as write_file:
                    json.dump(safe_info, write_file)
                return 'Too much errors occured, safe point was made, try in 40 minutes'
        for item in search_results['response']['items']:
            all_schoolers[year][sex].append(item['id'])
            q += 1
        if len(all_schoolers[year][sex]) == 0:
            test_search = users_search(school=946, birth_year=1996, sex=1).json()
            while 'error' in test_search:
                print('Error occured')
                print(test_search)
                time.sleep(3)
                test_search = users_search(school=946, birth_year=1996, sex=1).json()
            if test_search['response']['count'] == 0:
                safe_info = {'school': school, 'year': year, 'sex': sex}
                file_name = "schoolers_safe_point" + str(school) + ".json" 
                with open(file_name, "w") as write_file:
                    json.dump(all_schoolers, write_file)
                with open('start_file', "w") as write_file:
                    json.dump(safe_info, write_file)
                return 'Search problem occured, safe point was made'
        print(params)
        print(f'найдено школьничков {q}')
    file_name = "schoolers_final" + str(school) + ".json" 
    with open(file_name, "w") as write_file:
        json.dump(all_schoolers, write_file)        
    return f'Search finished for school {str(school)}'

In [33]:
for school in top10_schools:
    result = search_for_schoolers(school, search_params, param_ind=0)
    print(result)

(1996, 1)
найдено школьничков 2
(1996, 2)
найдено школьничков 3
(1997, 1)
найдено школьничков 4
(1997, 2)
найдено школьничков 4
(1998, 1)
найдено школьничков 1
(1998, 2)
найдено школьничков 3
(1999, 1)
найдено школьничков 2
(1999, 2)
найдено школьничков 6
(2000, 1)
найдено школьничков 1
(2000, 2)
найдено школьничков 2
(2001, 1)
найдено школьничков 0
(2001, 2)
найдено школьничков 4
(2002, 1)
найдено школьничков 0
(2002, 2)
найдено школьничков 4
(2003, 1)
найдено школьничков 0
(2003, 2)
найдено школьничков 8
(2004, 1)
найдено школьничков 1
(2004, 2)
найдено школьничков 3
(2005, 1)
найдено школьничков 3
(2005, 2)
найдено школьничков 4
(2006, 1)
найдено школьничков 1
(2006, 2)
найдено школьничков 0
Search finished for school 1712330
(1996, 1)
найдено школьничков 37
(1996, 2)
найдено школьничков 23
(1997, 1)
найдено школьничков 34
(1997, 2)
найдено школьничков 25
(1998, 1)
найдено школьничков 37
(1998, 2)
найдено школьничков 22
(1999, 1)
найдено школьничков 28
(1999, 2)
найдено школьничков 

In [1]:
print(top10_schools)

NameError: name 'top10_schools' is not defined